In [ ]:
# store array on local pc

file_path = "G:\Meine Ablage\KI_Projekt\center_points\center_array.npy"
# Save the array to a binary file
np.save(file_path.replace("\\","/"), reshaped_array_final)

file_path = "G:\Meine Ablage\KI_Projekt\center_points\center_array.npy"
loaded_array = np.load(file_path.replace("\\","/"))

In [ ]:
def resize_image(vegi):
    height, width, c = vegi.shape
    new_height = 100
    ratio = new_height / height 
    new_width = int(width * ratio)
    dimensions = (new_width, new_height)
    print(dimensions)
    resized_img = cv2.resize(vegi, dimensions, interpolation=cv2.INTER_LINEAR)
    print("New shape:      ", resized_img.shape)

    return resized_img

In [ ]:
# current values for borders they roughly fit to the real hsv values. I test it for brown onion, carrot, brown potato with
# white background, respectively
def mask_image(img_hsv, img_rgb, border_low=(8, 50, 75), border_high=(15, 255, 160)):
    #15, 23, 135
    mask = cv2.inRange(img_hsv, border_low, border_high)
    result = cv2.bitwise_and(img_rgb, img_rgb, mask=mask)

    #plt.subplot(1, 3, 1)
    #plt.imshow(img_rgb)
    #plt.subplot(1, 3, 2)
    #plt.imshow(mask, cmap='gray')
    #plt.subplot(1, 3, 3)
    #plt.imshow(result)
    #plt.show()
    return mask, result

In [ ]:
def draw_bounding_box(img, low_threshold, high_threshold):
    # the thresholds are used as gradients thresholds. 
    # Something like, gradients smaller than the low_threshold or between the thresholds are ignored 
    result = img.copy()
    gray_square_onion = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    thresh = cv2.threshold(gray_square_onion, low_threshold, high_threshold, cv2.THRESH_BINARY)[1]

    # get contours
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        # The idea is only keep bigger bounding boxes. Probably it presents the object
        if w > 22 and h > 0.5:
            cv2.rectangle(result, (x, y), (x+w, y+h), (100, 0, 255), 1)
            ROI = img[y:y+h, x:x+w]
            print(f"ROI:. Size of Bounding Box: w = {w} / h = {h}")
        else:
            continue
    show_image_plt(result)

In [ ]:
def plot_color_space_rgb_to_hsv(img_rgb, convert_to_hsv=True):
    copy_rgb = img_rgb.copy()
    img_hsv = copy_rgb
    x_label = "R (X)"
    y_label = "G (Y)"
    z_label = "B (Z)"

    if convert_to_hsv:
        img_hsv = cv2.cvtColor(copy_rgb, cv2.COLOR_RGB2HSV)
        x_label = "Hue X"
        y_label = "Saturation Y"
        z_label = "Value Z"

    pixel_colors = copy_rgb.reshape((np.shape(copy_rgb)[0]*np.shape(copy_rgb)[1], 3))
    norm = colors.Normalize(vmin=-1.,vmax=1.)
    norm.autoscale(pixel_colors)
    pixel_colors = norm(pixel_colors).tolist()

    x, y, z = cv2.split(img_hsv)
    fig = plt.figure()
    axis = fig.add_subplot(1, 1, 1, projection="3d")

    axis.scatter(x.flatten(), y.flatten(), z.flatten(), facecolors=pixel_colors, marker=".")
    axis.set_xlabel(x_label)
    axis.set_ylabel(y_label)
    axis.set_zlabel(z_label)
    plt.show()

In [ ]:
def kMeans_(img_rgb, number_of_clusters, attempts):
    # KMeans Processing
    img_hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
    K = number_of_clusters
    onion_2D = img_hsv.reshape((-1,3)) #Reshape to 2 dimensional image (65536, 3)
    onion_2D = np.float32(onion_2D)

    CONSTANT_1 = 10
    CONSTANT_2 = 1.0
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, CONSTANT_1, CONSTANT_2)

    ret, label, center=cv2.kmeans(onion_2D, K, None, criteria, attempts, cv2.KMEANS_PP_CENTERS)
    center = np.uint8(center)
    res = center[label.flatten()]
    result_image = res.reshape((img_rgb.shape))

    return result_image, label, center

In [ ]:
def get_object_color(center, cluster_img_hsv):
    for point in center:
        edge_point = 0
        if (point == cluster_img_hsv[0][0]).all():
            edge_point += 1

        if (point == cluster_img_hsv[0][255]).all():
            edge_point += 1

        if (point == cluster_img_hsv[255][255]).all():
            edge_point += 1

        if (point == cluster_img_hsv[255][0]).all():
            edge_point += 1

        if edge_point >= 3:
            #print(f"Color Background found: {point}")
            pass

        if edge_point < 3:
            #print(f"Color Object found: {point}")
            return point

In [ ]:
#Expect gray img
def min_area_rect(img_rgb, low_thresh, high_thresh):
    # Convert the image to grayscale
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)
    #gray = img_rgb.copy()
    #gray = cv2.blur(gray, (3,3))

    #plt.imshow(gray, cmap='gray')
    #plt.show()

    # Use Otsu's method to calculate a dynamic threshold
    #a, thresh = cv2.threshold(gray, low_thresh, high_thresh, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    #print(thresh.min(), thresh.max())
    #thresh_low = 0.5 * thresh
    #thresh_high = 1.5 * thresh
    # Apply edge detection using the dynamic threshold
    edges = cv2.Canny(gray, low_thresh, high_thresh)
    
    # Find contours in the edged image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    print('Length Contours: ', len(contours))
    # Find the contour with the maximum area
    max_contour = max(contours, key=cv2.contourArea)
    #print(contours)
    # Get the rotated bounding box of the contour
    rect = cv2.minAreaRect(max_contour)
    box = cv2.boxPoints(rect)
    box = np.intp(box)
    
    # Draw the rotated bounding box on the original image
    result_image = img_rgb.copy()
    cv2.drawContours(result_image, [box], 0, (0, 255, 0), 2)
    thres = 10
    return result_image, thres, box

Own algorithm to calculate the color in the image. It calculates the mean value from the H, S and V values. 

In [ ]:
def calculate_object_color(vegi_without_green_rgb):
    # Calculate mean color value in HSV Color Space
    # Only keep pixels >(1,1,1) (black)
    # Only keep pixel > (1, 100, 100)
    # Only keep pixels < (255, 225, 225)
    # Filter by Saturation and Value
    vegi_without_green_hsv =  cv2.cvtColor(vegi_without_green_rgb, cv2.COLOR_RGB2HSV)
    h, s, v = cv2.split(vegi_without_green_hsv)
    h_1d = h.flatten()
    s_1d = s.flatten()
    v_1d = v.flatten()

    h_mean_counter = 0
    s_mean_counter = 0
    v_mean_counter = 0
    sum_h = 0
    sum_s = 0
    sum_v = 0
    for i in range(0, h_1d.shape[0]):
        if h_1d[i] > 1 and h_1d[i] < 250:
            sum_h += h_1d[i]
            h_mean_counter += 1

        if s_1d[i] > 100 and s_1d[i] < 225:
            sum_s += s_1d[i]
            s_mean_counter += 1
            sum_v += v_1d[i]
            v_mean_counter += 1
        #if v_1d[i] > 30 and v_1d[i] < 225:
        #    sum_v += v_1d[i]
        #    v_mean_counter += 1
    
    #Maybe catch exception if the incoming img has no object and because of that we devide by zero
    mean_h = int(sum_h / h_mean_counter)
    mean_s = int(sum_s / s_mean_counter)
    mean_v = int(sum_v / v_mean_counter)
    
    hsv_color = [mean_h, mean_s, mean_v]
   
    rgb_color = cv2.cvtColor(np.uint8([[hsv_color]]), cv2.COLOR_HSV2RGB)[0][0]
    
    return rgb_color